# Lizard

In [ ]:
pic_dir = r'D:/Desktop/5243Bianca/Final Project/Dorsal/Dorsal'

In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import shutil

def prepare_dataset(input_dir, output_dir, test_size=0.2):
    os.makedirs(os.path.join(output_dir, 'train', 'lizard'), exist_ok=True)
    os.makedirs(os.path.join(output_dir, 'train', 'background'), exist_ok=True)
    os.makedirs(os.path.join(output_dir, 'test', 'lizard'), exist_ok=True)
    os.makedirs(os.path.join(output_dir, 'test', 'background'), exist_ok=True)
    
    images = [f for f in os.listdir(input_dir) if f.endswith(('.jpg', '.jpeg', '.png','JPG'))]
    train_images, test_images = train_test_split(images, test_size=test_size, random_state=42)
    
    process_image_set(input_dir, train_images, os.path.join(output_dir, 'train'))
    process_image_set(input_dir, test_images, os.path.join(output_dir, 'test'))
    
    print(f"Dataset preparation complete. {len(train_images)} training images and {len(test_images)} testing images.")

def process_image_set(input_dir, image_list, output_dir):
    for img_name in image_list:
        img_path = os.path.join(input_dir, img_name)
        img = cv2.imread(img_path)
        if img is None:
            print(f"Warning: Could not read {img_path}")
            continue
        
        corrected_img = color_correction(img)
        lizard_mask, lizard_img = extract_lizard(corrected_img)
        
        lizard_output_path = os.path.join(output_dir, 'lizard', img_name)
        cv2.imwrite(lizard_output_path, lizard_img)
        
        background_img = create_background_image(corrected_img, lizard_mask)
        background_output_path = os.path.join(output_dir, 'background', f"bg_{img_name}")
        cv2.imwrite(background_output_path, background_img)

def color_correction(img):
    height, width = img.shape[:2]
    colorchecker_roi = img[height//4:height//2, width//2:width-width//10]
    result = img.copy()
    gray_patch = colorchecker_roi[colorchecker_roi.shape[0]//2, colorchecker_roi.shape[1]//2]
    
    if np.max(gray_patch) > 0:
        scale = 128 / np.mean(gray_patch)
        result = cv2.convertScaleAbs(img, alpha=scale, beta=0)
    
    return result

def extract_lizard(img):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    lower_lizard = np.array([20, 30, 50])
    upper_lizard = np.array([40, 255, 255])
    
    lizard_mask = cv2.inRange(hsv, lower_lizard, upper_lizard)
    kernel = np.ones((5, 5), np.uint8)
    lizard_mask = cv2.morphologyEx(lizard_mask, cv2.MORPH_OPEN, kernel)
    lizard_mask = cv2.morphologyEx(lizard_mask, cv2.MORPH_CLOSE, kernel)
    
    contours, _ = cv2.findContours(lizard_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if contours:
        max_contour = max(contours, key=cv2.contourArea)
        refined_mask = np.zeros_like(lizard_mask)
        cv2.drawContours(refined_mask, [max_contour], 0, 255, -1)
        lizard_mask = refined_mask
    
    lizard_img = img.copy()
    background = np.ones_like(img) * 255
    lizard_img = np.where(lizard_mask[:, :, np.newaxis] == 255, img, background)
    
    return lizard_mask, lizard_img

def create_background_image(img, lizard_mask):
    background_img = img.copy()
    background_img[lizard_mask == 255] = [0, 0, 0]
    return background_img

def augment_dataset(input_dir, output_dir, n_augmentations=5):
    import imgaug.augmenters as iaa
    os.makedirs(output_dir, exist_ok=True)
    
    aug_seq = iaa.Sequential([
        iaa.Fliplr(0.5),
        iaa.Affine(
            rotate=(-15, 15),
            scale=(0.8, 1.2),
            translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)}
        ),
        iaa.GaussianBlur(sigma=(0, 0.5)),
        iaa.AdditiveGaussianNoise(scale=(0, 0.05*255))
    ])
    
    images = [f for f in os.listdir(input_dir) if f.endswith(('.jpg', '.jpeg', '.png'))]
    
    for img_name in images:
        img_path = os.path.join(input_dir, img_name)
        img = cv2.imread(img_path)
        if img is None:
            continue
            
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        for i in range(n_augmentations):
            aug_img = aug_seq(image=img_rgb)
            aug_img_bgr = cv2.cvtColor(aug_img, cv2.COLOR_RGB2BGR)
            output_path = os.path.join(output_dir, f"aug_{i}_{img_name}")
            cv2.imwrite(output_path, aug_img_bgr)
    
    print(f"Created {len(images) * n_augmentations} augmented images.")

def main():
    input_dir = "D:/Desktop/5243Bianca/Final Project/Dorsal/Dorsal"
    processed_dir = "D:/Desktop/5243Bianca/Final Project/Dorsal/processed_data/"
    augmented_dir = "D:/Desktop/5243Bianca/Final Project/Dorsal/augmented_data/"
    
    prepare_dataset(input_dir, processed_dir)
    augment_dataset(os.path.join(processed_dir, 'train', 'lizard'), 
                    os.path.join(augmented_dir, 'lizard'))

if __name__ == "__main__":
    main()


Dataset preparation complete. 56 training images and 14 testing images.
Created 0 augmented images.


In [2]:

input_dir = "D:/Desktop/5243Bianca/Final Project/Dorsal/Dorsal"  # 包含原始图像的目录
processed_dir = "D:/Desktop/5243Bianca/Final Project/Dorsal/processed_data/"  # 处理后的图像目录
augmented_dir = "D:/Desktop/5243Bianca/Final Project/Dorsal/augmented_data/"  # 增强后的图像目录
prepare_dataset(input_dir, processed_dir)


Dataset preparation complete. 56 training images and 14 testing images.


In [58]:
import os
from PIL import Image

# Define source and destination directories
source_dir = r"D:/Desktop/5243Bianca/Final Project/Dorsal/Dorsal"
destination_dir = r"D:/Desktop/5243Bianca/Final Project/middle_section"

# Create destination directory if it doesn't exist
if not os.path.exists(destination_dir):
    os.makedirs(destination_dir)
    print(f"Created directory: {destination_dir}")

# Process all JPG files in the source directory
def process_images():
    count = 0
    # List all files in source directory
    for filename in os.listdir(source_dir):
        # Check if the file is a JPG
        if filename.lower().endswith(('.jpg', '.jpeg')):
            try:
                # Construct full file paths
                source_path = os.path.join(source_dir, filename)
                destination_path = os.path.join(destination_dir, filename)
                
                # Open and process the image
                with Image.open(source_path) as img:
                    # Get dimensions
                    width, height = img.size
                    
                    # Calculate crop boundaries
                    # Remove the left 1/4 and right 1/2
                    left_boundary = width // 4  # Left 1/4 removed
                    right_boundary = width // 2  # Keep only up to the middle (removing right 1/2)
                    
                    # Crop to keep only the middle section
                    middle_section = img.crop((left_boundary, 0, right_boundary, height))
                    
                    # Save the cropped image
                    middle_section.save(destination_path)
                    count += 1
                    print(f"Processed: {filename}")
            except Exception as e:
                print(f"Error processing {filename}: {e}")
    
    print(f"\nCompleted! {count} images were processed.")

if __name__ == "__main__":
    process_images()

Created directory: D:/Desktop/5243Bianca/Final Project/middle_section
Processed: IMG_1411.JPG
Processed: IMG_1414.JPG
Processed: IMG_1417.JPG
Processed: IMG_1419.JPG
Processed: IMG_1421.JPG
Processed: IMG_1423.JPG
Processed: IMG_1425.JPG
Processed: IMG_1427.JPG
Processed: IMG_1429.JPG
Processed: IMG_1432.JPG
Processed: IMG_1434.JPG
Processed: IMG_1436.JPG
Processed: IMG_1438.JPG
Processed: IMG_1440.JPG
Processed: IMG_1443.JPG
Processed: IMG_1446.JPG
Processed: IMG_1448.JPG
Processed: IMG_1451.JPG
Processed: IMG_1453.JPG
Processed: IMG_1455.JPG
Processed: IMG_1457.JPG
Processed: IMG_1459.JPG
Processed: IMG_1461.JPG
Processed: IMG_1463.JPG
Processed: IMG_1465.JPG
Processed: IMG_1468.JPG
Processed: P1010185.JPG
Processed: P1010187.JPG
Processed: P1010189.JPG
Processed: P1010191.JPG
Processed: P1010193.JPG
Processed: P1010195.JPG
Processed: P1010197.JPG
Processed: P1010199.JPG
Processed: P1010201.JPG
Processed: P1010203.JPG
Processed: P1010205.JPG
Processed: P1010207.JPG
Processed: P101020

In [5]:

import torch
import torchvision.transforms as transforms
from torchvision import models
from PIL import Image
import os
torch.cuda.empty_cache()

# 设置设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 加载 MobileNet 预训练模型
model = models.mobilenet_v2(pretrained=True)
model.eval()
model.to(device)

# 加载 ImageNet 类别标签
imagenet_classes_path = 'imagenet_classes.txt'
if not os.path.exists(imagenet_classes_path):
    import urllib.request
    url = 'https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt'
    urllib.request.urlretrieve(url, imagenet_classes_path)

with open(imagenet_classes_path) as f:
    classes = [line.strip() for line in f.readlines()]

# 图像预处理
def preprocess_image(image_path):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    image = Image.open(image_path).convert("RGB")
    image = transform(image).unsqueeze(0).to(device)
    return image

# 分类图像
def classify_image(image_path):
    try:
        with torch.no_grad():
            image = preprocess_image(image_path)
            outputs = model(image)
            _, predicted = outputs.max(1)
            print(f"Image {os.path.basename(image_path)} - Predicted Class: {predicted.item()}")
            torch.cuda.empty_cache()  # 每次推理后清理显存
    except Exception as e:
        print(f"Error processing {image_path}: {e}")

# 扫描指定目录中的所有 JPG 图像
def classify_images_in_directory(directory_path):
    if not os.path.exists(directory_path):
        print("Directory not found.")
        return
    
    image_files = [f for f in os.listdir(directory_path) if f.lower().endswith('.jpg')]
    if not image_files:
        print("No JPG images found.")
        return

    print(f"Found {len(image_files)} images.")
    for img_file in image_files:
        classify_image(os.path.join(directory_path, img_file))

# 运行分类
directory_path = r"D:/Desktop/5243Bianca/Final Project/middle_section"
classify_images_in_directory(directory_path)


Found 70 images.
Image IMG_1411.JPG - Predicted Class: 38
Image IMG_1414.JPG - Predicted Class: 38
Image IMG_1417.JPG - Predicted Class: 38
Image IMG_1419.JPG - Predicted Class: 38
Image IMG_1421.JPG - Predicted Class: 38
Image IMG_1423.JPG - Predicted Class: 38
Image IMG_1425.JPG - Predicted Class: 38
Image IMG_1427.JPG - Predicted Class: 38
Image IMG_1429.JPG - Predicted Class: 327
Image IMG_1432.JPG - Predicted Class: 38
Image IMG_1434.JPG - Predicted Class: 327
Image IMG_1436.JPG - Predicted Class: 38
Image IMG_1438.JPG - Predicted Class: 38
Image IMG_1440.JPG - Predicted Class: 38
Image IMG_1443.JPG - Predicted Class: 38
Image IMG_1446.JPG - Predicted Class: 38
Image IMG_1448.JPG - Predicted Class: 38
Image IMG_1451.JPG - Predicted Class: 38
Image IMG_1453.JPG - Predicted Class: 38
Image IMG_1455.JPG - Predicted Class: 38
Image IMG_1457.JPG - Predicted Class: 38
Image IMG_1459.JPG - Predicted Class: 38
Image IMG_1461.JPG - Predicted Class: 38
Image IMG_1463.JPG - Predicted Class: 

# SAM(Segment Anything Model) zero shot -> lizard mask -google colab

# Apply mask 

In [1]:
import os
import cv2
import numpy as np
from tqdm import tqdm

def extract_lizard(original_img_path, mask_img_path, output_path):
    """
    Extract lizard from original image using the mask and save with transparent background
    
    Args:
        original_img_path: Path to the original JPG image
        mask_img_path: Path to the binary mask PNG image
        output_path: Path to save the extracted lizard with transparent background
    """
    # Read the original image and mask
    original = cv2.imread(original_img_path)
    mask = cv2.imread(mask_img_path, cv2.IMREAD_GRAYSCALE)
    
    # Check if images were loaded properly
    if original is None:
        print(f"Error: Could not load original image {original_img_path}")
        return
    if mask is None:
        print(f"Error: Could not load mask image {mask_img_path}")
        return
    
    # Resize mask if dimensions don't match
    if original.shape[:2] != mask.shape[:2]:
        mask = cv2.resize(mask, (original.shape[1], original.shape[0]))
    
    # Create an alpha channel (transparency) from the mask
    # Convert to binary mask if not already (values 0 or 255)
    _, binary_mask = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)
    
    # Convert BGR to BGRA (add alpha channel)
    rgba = cv2.cvtColor(original, cv2.COLOR_BGR2BGRA)
    
    # Set alpha channel based on mask (255 for lizard, 0 for background)
    rgba[:, :, 3] = binary_mask
    
    # Save the result with transparency
    cv2.imwrite(output_path, rgba)
    
def process_all_images(original_dir, mask_dir, output_dir):
    """
    Process all images in the directories
    
    Args:
        original_dir: Directory containing original JPG images
        mask_dir: Directory containing mask PNG images
        output_dir: Directory to save output images
    """
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Get list of mask files
    mask_files = [f for f in os.listdir(mask_dir) if f.endswith('_mask.png')]
    
    for mask_file in tqdm(mask_files, desc="Processing images"):
        # Extract base name from mask (remove _mask.png)
        base_name = mask_file.replace('_raw_mask.png', '')
        
        # Find corresponding original file
        original_file = f"{base_name}.JPG"
        
        # Build full paths
        original_path = os.path.join(original_dir, original_file)
        mask_path = os.path.join(mask_dir, mask_file)
        output_path = os.path.join(output_dir, f"{base_name}_extracted.png")
        
        # Check if original file exists
        if os.path.exists(original_path):
            extract_lizard(original_path, mask_path, output_path)
        else:
            # Try with lowercase extension
            original_file = f"{base_name}.jpg"
            original_path = os.path.join(original_dir, original_file)
            if os.path.exists(original_path):
                extract_lizard(original_path, mask_path, output_path)
            else:
                print(f"Warning: Could not find original image for mask {mask_file}")

if __name__ == "__main__":
    # Directory paths - update these according to your file structure
    original_dir = "D:/Desktop/5243Bianca/Final Project/Dorsal/Dorsal"  # Where the original JPG files are located
    mask_dir = "D:/Desktop/5243Bianca/Final Project/dorsal_mask"  # Where the mask PNG files are located
    output_dir = "D:/Desktop/5243Bianca/Final Project/extracted_lizards"  # Where to save the extracted lizards
    
    process_all_images(original_dir, mask_dir, output_dir)
    print(f"Finished! Extracted lizards saved to {output_dir}")

Processing images: 100%|██████████| 70/70 [00:53<00:00,  1.31it/s]

Finished! Extracted lizards saved to D:/Desktop/5243Bianca/Final Project/extracted_lizards


# Data augmentation

In [ ]:
# D:\Desktop\5243Bianca\Final Project\extracted_lizards

In [ ]:
import tensorflow as tf
import os
import numpy as np
from PIL import Image
import cv2
from tqdm import tqdm

input_dir = "D:/Desktop/5243Bianca/Final Project/extracted_lizards"
output_dir = "D:/Desktop/5243Bianca/Final Project/augmented_lizards"

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

def augment_transparent_image(image_path, output_dir, num_augmentations=10):

    img = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
    if img is None:
        print(f"无法读取图像: {image_path}")
        return
    
    filename = os.path.basename(image_path)
    base_name = os.path.splitext(filename)[0]
    
    rgb = img[:,:,:3]
    alpha = img[:,:,3] if img.shape[2] == 4 else np.ones(img.shape[:2], dtype=np.uint8) * 255
    
    max_rotation = 20
    max_shift = 0.2
    max_zoom = 0.2
    
    for i in range(num_augmentations):
        h, w = img.shape[:2]
        
        angle = np.random.uniform(-max_rotation, max_rotation)
        rotation_matrix = cv2.getRotationMatrix2D((w/2, h/2), angle, 1)
        
        tx = np.random.uniform(-max_shift, max_shift) * w
        ty = np.random.uniform(-max_shift, max_shift) * h
        rotation_matrix[0, 2] += tx
        rotation_matrix[1, 2] += ty

        scale = np.random.uniform(1-max_zoom, 1+max_zoom)
        scaled_w, scaled_h = int(w * scale), int(h * scale)
        
        rgb_transformed = cv2.warpAffine(
            rgb, rotation_matrix, (w, h), 
            flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT, borderValue=(0,0,0)
        )
        
        alpha_transformed = cv2.warpAffine(
            alpha, rotation_matrix, (w, h), 
            flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT, borderValue=0
        )
        
        if np.random.random() > 0.5:
            rgb_transformed = cv2.flip(rgb_transformed, 1)
            alpha_transformed = cv2.flip(alpha_transformed, 1)

        augmented_img = np.zeros((h, w, 4), dtype=np.uint8)
        augmented_img[:,:,:3] = rgb_transformed
        augmented_img[:,:,3] = alpha_transformed

        output_path = os.path.join(output_dir, f"{base_name}_aug_{i}.png")
        cv2.imwrite(output_path, augmented_img)

image_files = [f for f in os.listdir(input_dir) if f.endswith('.png')]

for filename in tqdm(image_files, desc="增强图像"):
    image_path = os.path.join(input_dir, filename)
    augment_transparent_image(image_path, output_dir)

print("数据增强完成！")

增强图像: 100%|██████████| 66/66 [07:56<00:00,  7.23s/it]

数据增强完成！


# feature extraction，t-SNE,RF

## augmentation

In [10]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.ensemble import RandomForestClassifier
from sklearn.manifold import TSNE
from tqdm import tqdm
from tensorflow.keras.applications import MobileNetV2, MobileNetV3Small
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.layers import GlobalAveragePooling2D

# Set paths
augmented_dir = "D:/Desktop/5243Bianca/Final Project/augmented_lizards"
original_dir = "D:/Desktop/5243Bianca/Final Project/extracted_lizards"
features_output_dir = "D:/Desktop/5243Bianca/Final Project/lizard_features"
visualization_dir = "D:/Desktop/5243Bianca/Final Project/visualizations"
class_mapping_file = "D:/Desktop/5243Bianca/Final Project/filename_class.xlsx"

# Create directories if they don't exist
for directory in [features_output_dir, visualization_dir]:
    if not os.path.exists(directory):
        os.makedirs(directory)

# Load the class mapping file
def load_class_mapping():
    try:
        # 指定 header=0 表示第一行是列名（类名）
        mapping_df = pd.read_excel(class_mapping_file, header=0)
        class_names = mapping_df.columns.tolist()
        
        filename_to_class = {}
        
        for class_name in class_names:
            filenames = mapping_df[class_name].dropna().tolist()
            for filename in filenames:
                # 去掉空格和扩展名（如 .JPG）
                filename_clean = os.path.splitext(str(filename).strip())[0].strip()
                filename_to_class[filename_clean] = class_name
        
        return filename_to_class, class_names
    except Exception as e:
        print(f"Error loading class mapping file: {e}")
        return {}, []


# Function to load and preprocess an image for MobileNet models
def load_and_preprocess_image(image_path, target_size=(224, 224)):
    # Load image with PIL to properly handle transparency
    img = Image.open(image_path).convert('RGBA')
    
    # Create a white background image
    background = Image.new('RGBA', img.size, (255, 255, 255, 255))
    
    # Alpha composite the image with the white background
    img = Image.alpha_composite(background, img)
    
    # Convert to RGB (remove alpha channel)
    img = img.convert('RGB')
    
    # Resize image
    img = img.resize(target_size)
    
    # Convert to array and preprocess
    img_array = img_to_array(img)
    
    # Scale pixel values to [0,1]
    img_array = img_array / 255.0
    
    return img_array

# Create feature extractors based on pretrained models
def create_feature_extractors():
    # MobileNetV2
    base_model_v2 = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    x = base_model_v2.output
    x = GlobalAveragePooling2D()(x)
    model_v2 = Model(inputs=base_model_v2.input, outputs=x)
    
    # Freeze all layers in the base model
    for layer in base_model_v2.layers:
        layer.trainable = False
    
    # MobileNetV3Small
    base_model_v3 = MobileNetV3Small(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    x = base_model_v3.output
    x = GlobalAveragePooling2D()(x)
    model_v3 = Model(inputs=base_model_v3.input, outputs=x)
    
    # Freeze all layers in the base model
    for layer in base_model_v3.layers:
        layer.trainable = False
    
    return {
        'mobilenetv2': model_v2,
        'mobilenetv3': model_v3,
    }

import re

def get_original_filename(filename):
    """
    提取增强图像对应的原始文件名
    例子：
        IMG_1411_extracted_aug_0.png -> IMG_1411
        P1010185_extracted_aug_3.png -> P1010185
    """
    match = re.match(r"^(.*?)(?:_extracted)?(?:_aug_\d+)?\.png$", filename)
    if match:
        return match.group(1)
    return None


# Extract features from images
def extract_features(image_dir, feature_extractors, filename_to_class):
    image_paths = []
    labels = []
    
    # Collect all image paths and extract labels from filenames
    for filename in os.listdir(image_dir):
        if filename.endswith('.png'):
            # Extract the original filename (before augmentation)
            # Format is "种群名_extracted_aug_数字.png"
            # 提取原始文件名
            original_name = get_original_filename(filename)

            # 匹配 class
            class_name = filename_to_class.get(original_name)
            if class_name:
                image_paths.append(os.path.join(image_dir, filename))
                labels.append(class_name)
            else:
                print(f"Warning: Could not determine class for {filename}")
    
    features = {}
    for model_name, model in feature_extractors.items():
        print(f"Extracting features using {model_name}...")
        model_features = []
        
        for img_path in tqdm(image_paths, desc=f"{model_name} feature extraction"):
            try:
                img_array = load_and_preprocess_image(img_path)
                img_array = np.expand_dims(img_array, axis=0)
                features_vector = model.predict(img_array, verbose=0)
                model_features.append(features_vector.flatten())
            except Exception as e:
                print(f"Error processing {img_path}: {e}")
                # Add a placeholder of zeros to maintain index alignment
                model_features.append(np.zeros(model.output_shape[1:]))
        
        features[model_name] = np.array(model_features)
    
    return features, labels, image_paths

# Visualize features using t-SNE
def visualize_features_tsne(features, labels, model_name, class_names):
    print(f"Running t-SNE visualization for {model_name} features...")
    
    # Convert labels to numerical format for coloring
    label_to_id = {label: i for i, label in enumerate(class_names)}
    numeric_labels = [label_to_id[label] for label in labels]
    
    # Apply t-SNE for dimensionality reduction
    tsne = TSNE(n_components=2, random_state=42, perplexity=min(30, len(labels)-1))
    features_2d = tsne.fit_transform(features)
    
    # Create scatter plot
    plt.figure(figsize=(12, 10))
    scatter = plt.scatter(features_2d[:, 0], features_2d[:, 1], c=numeric_labels, cmap='viridis', alpha=0.7)
    
    # Add legend
    legend1 = plt.legend(scatter.legend_elements()[0], class_names, title="Lizard Types", loc="upper right")
    plt.gca().add_artist(legend1)
    
    plt.title(f't-SNE visualization of {model_name} features')
    plt.tight_layout()
    plt.savefig(os.path.join(visualization_dir, f'{model_name}_tsne.png'), dpi=300)
    plt.close()

# Use Random Forest to select important features
def select_features(features, labels, model_name, class_names, n_features=20):
    print(f"Selecting top {n_features} features from {model_name}...")
    
    # Convert labels to numerical
    label_to_id = {label: i for i, label in enumerate(class_names)}
    numeric_labels = np.array([label_to_id[label] for label in labels])
    
    # Train a Random Forest classifier to get feature importance
    rf = RandomForestClassifier(n_estimators=100, random_state=42)
    rf.fit(features, numeric_labels)
    
    # Get feature importance
    importances = rf.feature_importances_
    
    # Create indices of top n features
    indices = np.argsort(importances)[::-1][:n_features]
    top_importances = importances[indices]
    
    # Plot feature importance
    plt.figure(figsize=(10, 6))
    plt.title(f'Top {n_features} Feature Importances - {model_name}')
    plt.bar(range(n_features), top_importances)
    plt.xticks(range(n_features), indices, rotation=90)
    plt.tight_layout()
    plt.savefig(os.path.join(visualization_dir, f'{model_name}_feature_importance.png'), dpi=300)
    plt.close()
    
    # Return selected features and their indices
    return features[:, indices], indices, top_importances

# Main execution
def main():
    # Load class mapping from Excel
    filename_to_class, class_names = load_class_mapping()
    print("映射表中的文件名（去除扩展名）如下：")
    print(list(filename_to_class.keys()))

    print(f"Loaded class mapping for {len(filename_to_class)} files across {len(class_names)} lizard classes.")
    
    # Create feature extractors
    feature_extractors = create_feature_extractors()
    
    # Extract features from augmented images
    features, labels, image_paths = extract_features(augmented_dir, feature_extractors, filename_to_class)
    
    # Save the mapping of images to labels
    image_labels_df = pd.DataFrame({
        'image_path': image_paths,
        'class': labels
    })
    image_labels_df.to_csv(os.path.join(features_output_dir, 'image_labels.csv'), index=False)
    
    # Process each model's features
    for model_name, model_features in features.items():
        # Save raw features
        np.save(os.path.join(features_output_dir, f'{model_name}_features.npy'), model_features)
        np.save(os.path.join(features_output_dir, f'labels.npy'), np.array(labels))
        
        # Visualize with t-SNE
        visualize_features_tsne(model_features, labels, model_name, class_names)
        
        # Select top features
        selected_features, top_indices, top_importances = select_features(model_features, labels, model_name, class_names)
        
        # Save selected features
        np.save(os.path.join(features_output_dir, f'{model_name}_selected_features.npy'), selected_features)
        np.save(os.path.join(features_output_dir, f'{model_name}_top_indices.npy'), top_indices)
        
        # Create a DataFrame with feature importance info
        feature_info = pd.DataFrame({
            'feature_index': top_indices,
            'importance': top_importances
        })
        feature_info.to_csv(os.path.join(features_output_dir, f'{model_name}_feature_importance.csv'), index=False)

if __name__ == "__main__":
    main()
    print("特征提取、可视化与选择完成！")

映射表中的文件名（去除扩展名）如下：
['P1010185', 'P1010187', 'P1010189', 'P1010191', 'P1010193', 'P1010195', 'P1010197', 'P1010199', 'P1010201', 'P1010203', 'P1010205', 'P1010207', 'P1010209', 'P1010211', 'P1010330', 'P1010333', 'P1010336', 'P1010338', 'P1010340', 'P1010343', 'P1010346', 'P1010348', 'P1010350', 'P1010352', 'P1010355', 'P1010357', 'P1010360', 'IMG_1411', 'IMG_1414', 'IMG_1417', 'IMG_1419', 'IMG_1421', 'IMG_1423', 'IMG_1425', 'IMG_1427', 'IMG_1429', 'IMG_1432', 'IMG_1434', 'IMG_1436', 'IMG_1438', 'IMG_1440', 'IMG_1443', 'IMG_1446', 'IMG_1448', 'YQ1401-D', 'YQ1402-D', 'YQ1403-D', 'YQ1404-D', 'YQ1405-D', 'YQ1406-D', 'YQ1407-D', 'YQ1408-D', 'YQ1409-D', 'YQ1410-D', 'YQ1411-D', 'YQ1412-D', 'YQ1413-D', 'YQ1414-D', 'YQ1415-D', 'YQ1416-D', 'YQ1417-D', 'IMG_1451', 'IMG_1453', 'IMG_1455', 'IMG_1457', 'IMG_1459', 'IMG_1461', 'IMG_1463', 'IMG_1465', 'IMG_1468']
Loaded class mapping for 70 files across 4 lizard classes.
Extracting features using mobilenetv2...


mobilenetv2 feature extraction: 100%|██████████| 660/660 [04:57<00:00,  2.22it/s]


Extracting features using mobilenetv3...


mobilenetv3 feature extraction: 100%|██████████| 660/660 [04:42<00:00,  2.33it/s]


Running t-SNE visualization for mobilenetv2 features...
Selecting top 20 features from mobilenetv2...
Running t-SNE visualization for mobilenetv3 features...
Selecting top 20 features from mobilenetv3...
特征提取、可视化与选择完成！


In [21]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, KFold, GridSearchCV
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from skopt import BayesSearchCV
from skopt.space import Real, Integer, Categorical

# Set paths
features_output_dir = "D:/Desktop/5243Bianca/Final Project/lizard_features"
results_dir = "D:/Desktop/5243Bianca/Final Project/classification_results"

# Create results directory if it doesn't exist
if not os.path.exists(results_dir):
    os.makedirs(results_dir)

def load_features(model_name, use_selected=True):
    """
    Load features extracted from the specified model
    
    Args:
        model_name: 'mobilenetv2' or 'mobilenetv3'
        use_selected: Whether to use the pre-selected top features
    
    Returns:
        features: Feature matrix
        labels: Class labels
    """
    # Load labels
    labels = np.load(os.path.join(features_output_dir, 'labels.npy'), allow_pickle=True)
    
    # Load features
    if use_selected:
        features = np.load(os.path.join(features_output_dir, f'{model_name}_selected_features.npy'))
    else:
        features = np.load(os.path.join(features_output_dir, f'{model_name}_features.npy'))
    
    return features, labels

def encode_labels(labels):
    """Convert string labels to numerical values"""
    label_encoder = LabelEncoder()
    return label_encoder.fit_transform(labels), label_encoder

def train_and_evaluate_classifiers(features, labels):
    """
    Train and evaluate multiple classifiers using 4-fold cross-validation
    
    Args:
        features: Feature matrix
        labels: Numerical class labels
        
    Returns:
        results_df: DataFrame with performance metrics for each classifier
    """
    # Define classifiers to test
    classifiers = {
        'Linear SVM': SVC(kernel='linear', random_state=42),
        'RBF SVM': SVC(kernel='rbf', random_state=42),
        'Random Forest': RandomForestClassifier(random_state=42),
        'AdaBoost': AdaBoostClassifier(random_state=42),
        'KNN': KNeighborsClassifier(),
        'Naive Bayes': GaussianNB()
    }
    
    # Initialize results storage
    results = []
    
    # Initialize K-fold cross-validation
    kf = KFold(n_splits=4, shuffle=True, random_state=42)
    
    # Scale features
    scaler = MinMaxScaler()
    
    for clf_name, clf in classifiers.items():
        print(f"Training and evaluating {clf_name}...")
        
        fold_accuracies = []
        fold_f1_scores = []
        fold = 1
        
        # Perform cross-validation
        for train_idx, test_idx in kf.split(features):
            X_train, X_test = features[train_idx], features[test_idx]
            y_train, y_test = labels[train_idx], labels[test_idx]
            
            # Scale features
            X_train = scaler.fit_transform(X_train)
            X_test = scaler.transform(X_test)
            
            # Train classifier
            clf.fit(X_train, y_train)
            
            # Make predictions
            y_pred = clf.predict(X_test)
            
            # Compute metrics
            accuracy = accuracy_score(y_test, y_pred)
            f1 = f1_score(y_test, y_pred, average='weighted')
            
            fold_accuracies.append(accuracy)
            fold_f1_scores.append(f1)
            
            # Record individual fold results
            results.append({
                'Classifier': clf_name,
                'Fold': fold,
                'Accuracy': accuracy,
                'F1-Score': f1
            })
            
            fold += 1
        
        # Record average results
        results.append({
            'Classifier': clf_name,
            'Fold': 'Average',
            'Accuracy': np.mean(fold_accuracies),
            'F1-Score': np.mean(fold_f1_scores)
        })
    
    # Convert to DataFrame
    results_df = pd.DataFrame(results)
    
    return results_df

def plot_classifier_performance(results_df):
    """Plot the performance of different classifiers"""
    # Extract average performances
    avg_results = results_df[results_df['Fold'] == 'Average'].copy()
    
    # Create figure with subplots
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # Plot accuracy
    sns.barplot(x='Classifier', y='Accuracy', data=avg_results, ax=ax1)
    ax1.set_title('Average Accuracy Across 4-Fold CV')
    ax1.set_ylim(0, 1)  # Set y-axis from 0 to 1
    ax1.set_xticklabels(ax1.get_xticklabels(), rotation=45, ha='right')
    
    # Plot F1-Score
    sns.barplot(x='Classifier', y='F1-Score', data=avg_results, ax=ax2)
    ax2.set_title('Average F1-Score Across 4-Fold CV')
    ax2.set_ylim(0, 1)  # Set y-axis from 0 to 1
    ax2.set_xticklabels(ax2.get_xticklabels(), rotation=45, ha='right')
    
    plt.tight_layout()
    plt.savefig(os.path.join(results_dir, 'classifier_performance.png'), dpi=300)
    plt.close()
    
    return fig

def optimize_best_classifier(features, labels, best_classifier):
    """
    Perform Bayesian optimization for the best classifier
    
    Args:
        features: Feature matrix
        labels: Numerical class labels
        best_classifier: Name of the best classifier
        
    Returns:
        best_params: Best parameters found
        best_score: Best score achieved
    """
    # Scale features
    scaler = MinMaxScaler()
    scaled_features = scaler.fit_transform(features)
    
    # Define search spaces for different classifiers
    if best_classifier == 'Linear SVM':
        model = SVC(kernel='linear', random_state=42)
        param_space = {
            'C': Real(0.1, 10.0, prior='log-uniform'),
            'tol': Real(1e-5, 1e-2, prior='log-uniform')
        }
    elif best_classifier == 'RBF SVM':
        model = SVC(kernel='rbf', random_state=42)
        param_space = {
            'C': Real(0.1, 10.0, prior='log-uniform'),
            'gamma': Real(0.001, 1.0, prior='log-uniform'),
            'tol': Real(1e-5, 1e-2, prior='log-uniform')
        }
    elif best_classifier == 'Random Forest':
        model = RandomForestClassifier(random_state=42)
        param_space = {
            'n_estimators': Integer(50, 300),
            'max_depth': Integer(3, 20),
            'min_samples_split': Integer(2, 10),
            'min_samples_leaf': Integer(1, 5)
        }
    elif best_classifier == 'AdaBoost':
        model = AdaBoostClassifier(random_state=42)
        param_space = {
            'n_estimators': Integer(50, 300),
            'learning_rate': Real(0.01, 1.0, prior='log-uniform')
        }
    elif best_classifier == 'KNN':
        model = KNeighborsClassifier()
        param_space = {
            'n_neighbors': Integer(3, 15),
            'weights': Categorical(['uniform', 'distance']),
            'p': Integer(1, 2)
        }
    else:  # Naive Bayes
        model = GaussianNB()
        param_space = {
            'var_smoothing': Real(1e-10, 1e-8, prior='log-uniform')
        }
    
    # Define Bayesian search
    opt = BayesSearchCV(
        model,
        param_space,
        n_iter=30,  # Number of parameter settings that are sampled
        cv=4,        # 4-fold cross-validation
        n_jobs=-1,   # Use all available CPUs
        random_state=42,
        scoring='accuracy'
    )
    
    # Perform optimization
    print(f"\nPerforming Bayesian optimization for {best_classifier}...")
    opt.fit(scaled_features, labels)
    
    # Get results
    print(f"Best parameters: {opt.best_params_}")
    print(f"Best cross-validation score: {opt.best_score_:.4f}")
    
    # Train the model with best params on the full dataset
    best_model = opt.best_estimator_
    
    # Save optimization results
    optimization_results = {
        'best_params': opt.best_params_,
        'best_score': opt.best_score_,
        'all_scores': opt.cv_results_
    }
    
    # Evaluate the optimized model with cross-validation
    kf = KFold(n_splits=4, shuffle=True, random_state=42)
    fold_accuracies = []
    fold_f1_scores = []
    all_y_true = []
    all_y_pred = []
    
    for train_idx, test_idx in kf.split(scaled_features):
        X_train, X_test = scaled_features[train_idx], scaled_features[test_idx]
        y_train, y_test = labels[train_idx], labels[test_idx]
        
        # Train with optimal parameters
        best_model.fit(X_train, y_train)
        
        # Predict
        y_pred = best_model.predict(X_test)
        
        # Collect results
        all_y_true.extend(y_test)
        all_y_pred.extend(y_pred)
        
        # Metrics for this fold
        accuracy = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred, average='weighted')
        
        fold_accuracies.append(accuracy)
        fold_f1_scores.append(f1)
    
    # Final metrics
    final_accuracy = np.mean(fold_accuracies)
    final_f1 = np.mean(fold_f1_scores)
    
    # Create and plot confusion matrix
    cm = confusion_matrix(all_y_true, all_y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title(f'Confusion Matrix - Optimized {best_classifier}')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.tight_layout()
    plt.savefig(os.path.join(results_dir, f'confusion_matrix_{best_classifier.lower().replace(" ", "_")}.png'), dpi=300)
    plt.close()
    
    # Classification report
    report = classification_report(all_y_true, all_y_pred, output_dict=True)
    report_df = pd.DataFrame(report).transpose()
    report_df.to_csv(os.path.join(results_dir, f'classification_report_{best_classifier.lower().replace(" ", "_")}.csv'))
    
    print(f"\nFinal evaluation of optimized {best_classifier}:")
    print(f"Average Accuracy: {final_accuracy:.4f}")
    print(f"Average F1-Score: {final_f1:.4f}")
    
    return optimization_results

def compare_models_and_features():
    """Compare performance between MobileNetV2 and MobileNetV3 features"""
    models = ['mobilenetv2', 'mobilenetv3']
    feature_types = [True, False]  # True for selected features, False for all features
    
    all_results_list = []
    
    for model_name in models:
        for use_selected in feature_types:
            feature_type = "Selected" if use_selected else "All"
            print(f"\nEvaluating {model_name} - {feature_type} Features")
            
            # Load features and labels
            features, labels = load_features(model_name, use_selected)
            encoded_labels, _ = encode_labels(labels)
            
            # Train and evaluate
            model_results = train_and_evaluate_classifiers(features, encoded_labels)
            
            # Add model and feature type information
            model_results['Model'] = model_name
            model_results['Feature Type'] = feature_type
            
            # Append to overall results
            all_results_list.append(model_results)
    
    # Combine all results
    all_results = pd.concat(all_results_list)
    all_results.to_csv(os.path.join(results_dir, 'all_classification_results.csv'), index=False)
    
    # Plot comparison of average performances
    avg_results = all_results[all_results['Fold'] == 'Average'].copy()
    
    plt.figure(figsize=(15, 10))
    
    # Plot accuracy comparison
    plt.subplot(2, 1, 1)
    sns.barplot(x='Classifier', y='Accuracy', hue='Model', data=avg_results)
    plt.title('Average Accuracy Comparison')
    plt.ylim(0, 1)
    plt.legend(title='Model')
    plt.xticks(rotation=45, ha='right')
    
    # Plot F1-score comparison
    plt.subplot(2, 1, 2)
    sns.barplot(x='Classifier', y='F1-Score', hue='Model', data=avg_results)
    plt.title('Average F1-Score Comparison')
    plt.ylim(0, 1)
    plt.legend(title='Model')
    plt.xticks(rotation=45, ha='right')
    
    plt.tight_layout()
    plt.savefig(os.path.join(results_dir, 'model_comparison.png'), dpi=300)
    plt.close()
    
    # Find the best performing configuration - using numpy methods for reliability
    best_accuracy_idx = avg_results['Accuracy'].values.argmax()
    best_row = avg_results.iloc[best_accuracy_idx]
    
    # Create a dictionary to hold the best configuration
    best_config = {
        'Model': best_row['Model'],
        'Feature Type': best_row['Feature Type'],
        'Classifier': best_row['Classifier'],
        'Accuracy': best_row['Accuracy'],
        'F1-Score': best_row['F1-Score']
    }
    
    print("\nBest configuration:")
    print(f"Model: {best_config['Model']}")
    print(f"Feature Type: {best_config['Feature Type']}")
    print(f"Classifier: {best_config['Classifier']}")
    print(f"Accuracy: {best_config['Accuracy']:.4f}")
    print(f"F1-Score: {best_config['F1-Score']:.4f}")
    
    return best_config, all_results

def main():
    # Compare different models and feature types
    best_config, all_results = compare_models_and_features()
    
    # Get the best model, feature type, and classifier
    best_model = best_config['Model']
    best_feature_type = best_config['Feature Type'] == 'Selected'
    best_classifier = best_config['Classifier']
    
    # Load the best features
    features, labels = load_features(best_model, best_feature_type)
    encoded_labels, label_encoder = encode_labels(labels)
    
    # Optimize the best classifier
    optimization_results = optimize_best_classifier(features, encoded_labels, best_classifier)
    
    # Save results
    summary = {
        'best_model': best_model,
        'best_feature_type': 'Selected' if best_feature_type else 'All',
        'best_classifier': best_classifier,
        'original_accuracy': best_config['Accuracy'],
        'original_f1': best_config['F1-Score'],
        'optimized_accuracy': optimization_results['best_score'],
        'optimized_params': optimization_results['best_params']
    }
    
    with open(os.path.join(results_dir, 'optimization_summary.txt'), 'w') as f:
        for key, value in summary.items():
            f.write(f"{key}: {value}\n")
    
    print("\nClassification and optimization completed successfully!")
    print(f"All results saved to: {results_dir}")

if __name__ == "__main__":
    main()


Evaluating mobilenetv2 - Selected Features
Training and evaluating Linear SVM...
Training and evaluating RBF SVM...
Training and evaluating Random Forest...
Training and evaluating AdaBoost...
Training and evaluating KNN...
Training and evaluating Naive Bayes...

Evaluating mobilenetv2 - All Features
Training and evaluating Linear SVM...
Training and evaluating RBF SVM...
Training and evaluating Random Forest...
Training and evaluating AdaBoost...
Training and evaluating KNN...
Training and evaluating Naive Bayes...

Evaluating mobilenetv3 - Selected Features
Training and evaluating Linear SVM...
Training and evaluating RBF SVM...
Training and evaluating Random Forest...
Training and evaluating AdaBoost...
Training and evaluating KNN...
Training and evaluating Naive Bayes...

Evaluating mobilenetv3 - All Features
Training and evaluating Linear SVM...
Training and evaluating RBF SVM...
Training and evaluating Random Forest...
Training and evaluating AdaBoost...
Training and evaluating 

In [ ]:
import os
import shutil
import pandas as pd

train_dir = "D:/Desktop/5243Bianca/Final Project/augmented_lizards"
excel_path = "D:/Desktop/5243Bianca/Final Project/filename_class.xlsx"
output_dir = "D:/Desktop/5243Bianca/Final Project/training_images"

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

df = pd.read_excel(excel_path, header=0)


for class_name in df.columns:

    prefix_list = df[class_name].dropna().astype(str).tolist()
    

    class_dir = os.path.join(output_dir, class_name)
    os.makedirs(class_dir, exist_ok=True)
    

    for prefix in prefix_list:
        for filename in os.listdir(train_dir):
            if filename.startswith(prefix):
                src_path = os.path.join(train_dir, filename)
                dst_path = os.path.join(class_dir, filename)
                
                # 拷贝文件
                shutil.copy2(src_path, dst_path)

print("图像分类完成，所有文件已重新整理到 training_images/ 中。")


图像分类完成，所有文件已重新整理到 training_images/ 中。


# pipeline

In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.svm import SVC
from PIL import Image
import joblib
import pickle

class LizardClassifier:
    def __init__(self, model_dir=None):
        """
        Initialize the lizard classifier pipeline with the best configuration.
        
        Args:
            model_dir: Directory containing saved model files (optional)
        """
        self.model_name = 'mobilenetv2'
        self.use_all_features = True  # Using all features instead of selected
        self.classifier_name = 'Linear SVM'
        
        # Create feature extractor (MobileNetV2)
        base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
        x = base_model.output
        x = GlobalAveragePooling2D()(x)
        self.feature_extractor = Model(inputs=base_model.input, outputs=x)
        
        # Freeze all layers in the base model
        for layer in base_model.layers:
            layer.trainable = False
        
        # Initialize classifier with optimized parameters
        self.classifier = SVC(
            kernel='linear',
            C=0.661009829541915,
            tol=0.0015246748254295628,
            random_state=42
        )
        
        # Initialize scaler for feature normalization
        self.scaler = MinMaxScaler()
        
        # Initialize label encoder
        self.label_encoder = LabelEncoder()
        
        # Load pre-trained components if model_dir is provided
        if model_dir:
            self.load_model(model_dir)
    
    def load_model(self, model_dir):
        """
        Load the pre-trained classifier and preprocessing components.
        
        Args:
            model_dir: Directory containing saved model files
        """
        # Load classifier
        self.classifier = joblib.load(os.path.join(model_dir, 'svm_classifier.joblib'))
        
        # Load scaler
        self.scaler = joblib.load(os.path.join(model_dir, 'feature_scaler.joblib'))
        
        # Load label encoder
        with open(os.path.join(model_dir, 'label_encoder.pkl'), 'rb') as f:
            self.label_encoder = pickle.load(f)
    
    def save_model(self, model_dir):
        """
        Save the trained classifier and preprocessing components.
        
        Args:
            model_dir: Directory to save model files
        """
        # Create directory if it doesn't exist
        if not os.path.exists(model_dir):
            os.makedirs(model_dir)
        
        # Save classifier
        joblib.dump(self.classifier, os.path.join(model_dir, 'svm_classifier.joblib'))
        
        # Save scaler
        joblib.dump(self.scaler, os.path.join(model_dir, 'feature_scaler.joblib'))
        
        # Save label encoder
        with open(os.path.join(model_dir, 'label_encoder.pkl'), 'wb') as f:
            pickle.dump(self.label_encoder, f)
    
    def preprocess_image(self, image_path):
        """
        Preprocess a single image for feature extraction.
        
        Args:
            image_path: Path to the image file
            
        Returns:
            Preprocessed image array
        """
        # Load image with PIL to properly handle transparency
        img = Image.open(image_path).convert('RGBA')
        
        # Create a white background image
        background = Image.new('RGBA', img.size, (255, 255, 255, 255))
        
        # Alpha composite the image with the white background
        img = Image.alpha_composite(background, img)
        
        # Convert to RGB (remove alpha channel)
        img = img.convert('RGB')
        
        # Resize image to match MobileNetV2 input size
        img = img.resize((224, 224))
        
        # Convert to array and normalize
        img_array = np.array(img).astype(np.float32)
        img_array = img_array / 255.0
        
        return img_array
    
    def extract_features(self, image_array):
        """
        Extract features from preprocessed image using MobileNetV2.
        
        Args:
            image_array: Preprocessed image array
            
        Returns:
            Feature vector
        """
        # Add batch dimension
        img_array = np.expand_dims(image_array, axis=0)
        
        # Extract features
        features = self.feature_extractor.predict(img_array, verbose=0)
        
        return features.flatten()
    
    def fit(self, image_paths, labels):
        """
        Train the classifier using a list of images and their labels.
        
        Args:
            image_paths: List of paths to images
            labels: List of class labels
        """
        print("Extracting features from training images...")
        features = []
        
        # Extract features from all images
        for img_path in image_paths:
            img_array = self.preprocess_image(img_path)
            features.append(self.extract_features(img_array))
        
        features = np.array(features)
        
        # Fit the label encoder
        encoded_labels = self.label_encoder.fit_transform(labels)
        
        # Scale features
        scaled_features = self.scaler.fit_transform(features)
        
        # Train the classifier
        print("Training SVM classifier...")
        self.classifier.fit(scaled_features, encoded_labels)
        
        print("Training completed successfully!")
    
    def predict(self, image_path):
        """
        Predict the class of a single image.
        
        Args:
            image_path: Path to the image file
            
        Returns:
            Predicted class label
        """
        # Preprocess image
        img_array = self.preprocess_image(image_path)
        
        # Extract features
        features = self.extract_features(img_array)
        
        # Scale features
        scaled_features = self.scaler.transform(features.reshape(1, -1))
        
        # Predict class
        encoded_prediction = self.classifier.predict(scaled_features)[0]
        
        # Convert to original label
        prediction = self.label_encoder.inverse_transform([encoded_prediction])[0]
        
        return prediction
    
    def predict_proba(self, image_path):
        """
        Predict class probabilities for a single image.
        
        Args:
            image_path: Path to the image file
            
        Returns:
            Dictionary mapping class labels to probabilities
        """
        # Preprocess image
        img_array = self.preprocess_image(image_path)
        
        # Extract features
        features = self.extract_features(img_array)
        
        # Scale features
        scaled_features = self.scaler.transform(features.reshape(1, -1))
        
        # Predict class probabilities
        if hasattr(self.classifier, 'predict_proba'):
            proba = self.classifier.predict_proba(scaled_features)[0]
        else:
            # For SVM, use decision_function and convert to probabilities
            decision = self.classifier.decision_function(scaled_features)
            proba = tf.nn.softmax(decision, axis=1).numpy()[0]
        
        # Create dictionary mapping class labels to probabilities
        classes = self.label_encoder.classes_
        class_proba = {classes[i]: proba[i] for i in range(len(classes))}
        
        return class_proba

# Example usage:
if __name__ == "__main__":
    # Initialize the pipeline
    classifier = LizardClassifier()
    
    # Example paths for model training
    images_dir = "D:/Desktop/5243Bianca/Final Project/training_images"
    model_save_dir = "D:/Desktop/5243Bianca/Final Project/save_model"
    
    # Example training code (commented out)
    
    # Load training data
    image_paths = []
    labels = []
    
    # Assuming folder structure with class names as subdirectories
    for class_name in os.listdir(images_dir):
        class_dir = os.path.join(images_dir, class_name)
        if os.path.isdir(class_dir):
            for filename in os.listdir(class_dir):
                if filename.endswith('.png') or filename.endswith('.jpg'):
                    img_path = os.path.join(class_dir, filename)
                    image_paths.append(img_path)
                    labels.append(class_name)
    
    # Train the classifier
    classifier.fit(image_paths, labels)
    
    # Save the trained model
    classifier.save_model(model_save_dir)

    
    # # Example prediction code

    # # Load a pre-trained model
    # classifier = LizardClassifier(model_dir=model_save_dir)
    
    # # Predict class for a new image
    # new_image_path = "path/to/new_lizard_image.png"
    # predicted_class = classifier.predict(new_image_path)
    # print(f"Predicted lizard class: {predicted_class}")
    
    # # Get class probabilities if needed
    # class_probabilities = classifier.predict_proba(new_image_path)
    # print("Class probabilities:")
    # for class_name, probability in class_probabilities.items():
    #     print(f"{class_name}: {probability:.4f}")
    

Extracting features from training images...
Training SVM classifier...
Training completed successfully!


# if want to classify a new image

In [6]:
import os
import cv2
import numpy as np
from tqdm import tqdm

def extract_lizard(original_img_path, mask_img_path, output_path):
    """
    Extract lizard from original image using the mask and save with transparent background
    
    Args:
        original_img_path: Path to the original JPG image
        mask_img_path: Path to the binary mask PNG image
        output_path: Path to save the extracted lizard with transparent background
    """
    # Read the original image and mask
    original = cv2.imread(original_img_path)
    mask = cv2.imread(mask_img_path, cv2.IMREAD_GRAYSCALE)
    
    # Check if images were loaded properly
    if original is None:
        print(f"Error: Could not load original image {original_img_path}")
        return
    if mask is None:
        print(f"Error: Could not load mask image {mask_img_path}")
        return
    
    # Resize mask if dimensions don't match
    if original.shape[:2] != mask.shape[:2]:
        mask = cv2.resize(mask, (original.shape[1], original.shape[0]))
    
    # Create an alpha channel (transparency) from the mask
    # Convert to binary mask if not already (values 0 or 255)
    _, binary_mask = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)
    
    # Convert BGR to BGRA (add alpha channel)
    rgba = cv2.cvtColor(original, cv2.COLOR_BGR2BGRA)
    
    # Set alpha channel based on mask (255 for lizard, 0 for background)
    rgba[:, :, 3] = binary_mask
    
    # Save the result with transparency
    cv2.imwrite(output_path, rgba)
    
def process_all_images(original_dir, mask_dir, output_dir):
    """
    Process all images in the directories
    
    Args:
        original_dir: Directory containing original JPG images
        mask_dir: Directory containing mask PNG images
        output_dir: Directory to save output images
    """
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Get list of mask files
    mask_files = [f for f in os.listdir(mask_dir) if f.endswith('_mask.png')]
    
    for mask_file in tqdm(mask_files, desc="Processing images"):
        # Extract base name from mask (remove _mask.png)
        base_name = mask_file.replace('_raw_mask.png', '')
        
        # Find corresponding original file
        original_file = f"{base_name}.JPG"
        
        # Build full paths
        original_path = os.path.join(original_dir, original_file)
        mask_path = os.path.join(mask_dir, mask_file)
        output_path = os.path.join(output_dir, f"{base_name}_extracted.png")
        
        # Check if original file exists
        if os.path.exists(original_path):
            extract_lizard(original_path, mask_path, output_path)
        else:
            # Try with lowercase extension
            original_file = f"{base_name}.jpg"
            original_path = os.path.join(original_dir, original_file)
            if os.path.exists(original_path):
                extract_lizard(original_path, mask_path, output_path)
            else:
                print(f"Warning: Could not find original image for mask {mask_file}")

if __name__ == "__main__":
    # Directory paths - update these according to your file structure
    original_dir = "D:/Desktop/5243Bianca/Final Project/Dorsal/Dorsal"  # Where the original JPG files are located
    mask_dir = "D:/Desktop/5243Bianca/Final Project/test_images/result"  # Where the mask PNG files are located
    output_dir = "D:/Desktop/5243Bianca/Final Project/test_images/extracted_lizards"  # Where to save the extracted lizards
    
    process_all_images(original_dir, mask_dir, output_dir)
    print(f"Finished! Extracted lizards saved to {output_dir}")

Processing images: 100%|██████████| 4/4 [00:02<00:00,  1.62it/s]

Finished! Extracted lizards saved to D:/Desktop/5243Bianca/Final Project/test_images/extracted_lizards


In [4]:
# Load the trained model
classifier = LizardClassifier(model_dir=model_save_dir)

# Predict class for a new image
new_image_path = "D:/Desktop/5243Bianca/Final Project/test_images/extracted_lizards/IMG_1417_extracted.png"
predicted_class = classifier.predict(new_image_path)
print(f"Predicted lizard class: {predicted_class}")

Predicted lizard class: YQ14


In [8]:
new_image_path = "D:/Desktop/5243Bianca/Final Project/test_images/extracted_lizards/IMG_1429_extracted.png"
predicted_class = classifier.predict(new_image_path)
print(f"Predicted lizard class: {predicted_class}")

Predicted lizard class: YQ14
